In [ ]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,update_topology,spanning_tree
import topo_meas_queries, process_network, linknet, update_topology

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic, platform_log_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
#model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123
#model_mrid="_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
#os.environ['GRIDAPPSD_SIMULATION_ID'] = opts.simulation_id
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'


gapps = GridAPPSD()
assert gapps.connected

In [ ]:
import switch_areas

In [ ]:
message = switch_areas.create_switch_areas(gapps, model_mrid)

In [ ]:
message = json.loads(message)

In [ ]:
message

In [ ]:
with open('output_message.json', 'w') as f:
    json.dump(message, f)

In [ ]:
[ConnNodeDict, EquipDict] = process_network.build_equip_dicts(gapps, model_mrid)


In [ ]:
BaseConnDict = json.dumps(ConnNodeDict)
MVConnNodeDict = json.loads(BaseConnDict)

In [ ]:
#[MVConnNodeDict, EquipDict] = process_network.build_equip_dicts(gapps, model_mrid)


In [ ]:
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine']
[ConnNodeDict, TerminalsDict, NodeList, TermList] = linknet.build_linknet_lists(ConnNodeDict, EquipDict, EqTypes)

In [ ]:
EqTypes = ['ACLineSegment', 'RatioTapChanger', 'SynchronousMachine']
[MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList] = linknet.build_linknet_lists(MVConnNodeDict, EquipDict, EqTypes)

In [ ]:
SwitchDict = {}
SwitchDict.update(EquipDict['Breaker'])
SwitchDict.update(EquipDict['Fuse'])
SwitchDict.update(EquipDict['LoadBreakSwitch'])
SwitchDict.update(EquipDict['Recloser'])
SwitchKeys = list(SwitchDict.keys())

In [ ]:
MVTrees = {}
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'Breaker', list(EquipDict['Breaker'].keys()), MVTrees, 'all')
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'Fuse', list(EquipDict['Fuse'].keys()), MVTrees, 'all')
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'LoadBreakSwitch', list(EquipDict['LoadBreakSwitch'].keys()), MVTrees, 'all')
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'Recloser', list(EquipDict['Recloser'].keys()), MVTrees, 'all')

MVBase = json.dumps(MVTrees)
MVBaseTree = json.loads(MVBase)

In [ ]:
DistAppStruct = {}
DistAppStruct['feeders'] = {}
DistAppStruct['feeders'][model_mrid] = {}
DistAppStruct['feeders'][model_mrid]['addressable_equipment'] = []
DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'] = []
DistAppStruct['feeders'][model_mrid]['connectivity_node'] = []

DistAppStruct['feeders'][model_mrid]['switch_areas'] = []
ProcessedNodes = []
XfmrDict = EquipDict['PowerTransformer']
XfmrDict.update(EquipDict['TransformerTank'])

SwitchKeys = list(MVTrees.keys())

for i1 in range(len(SwitchKeys)):
    SwitchArea = {}
    SwitchArea['boundary_switches'] = []
    SwitchArea['addressable_equipment'] = []
    SwitchArea['unaddressable_equipment'] = []
    SwitchArea['connectivity_node'] = []
    SwitchArea['secondary_areas'] = []
    DistArea = {}
    DistArea['distribution_transformer'] = []
    DistArea['addressable_equipment'] = []
    DistArea['unaddressable_equipment'] = []
    DistArea['connectivity_node'] = []
    DistAreaFlag1 = True
    #print(EquipDict['LoadBreakSwitch'][SwitchKeys[i1]]['name'])
    for i2 in range(len(MVTrees[SwitchKeys[i1]])):
        node = MVTrees[SwitchKeys[i1]][i2]
        
        print('mv node', ConnNodeDict[node]['name'], node)
        ProcessedNodes.append(node)
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Breaker'])
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Fuse'])
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['LoadBreakSwitch'])
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Recloser'])
#        SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['SynchronousMachine'])
 #       SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['PowerElectronicsConnection'])
  #      SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['LinearShuntCompensator'])
   #     SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['RatioTapChanger'])
    #    SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['ACLineSegment'])
     #   SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['PowerTransformer'])
      #  SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['TransformerTank'])
       # SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['Measurement'])
        SwitchArea['connectivity_node'].append(ConnNodeDict[node]['name'])
        DistArea = {}
        DistArea['distribution_transformer'] = []
        DistArea['addressable_equipment'] = []
        DistArea['unaddressable_equipment'] = []
        DistArea['connectivity_node'] = []
        DistXfmrs = ConnNodeDict[node]['TransformerTank'] #+ ConnNodeDict[node]['PowerTransformer']
        if DistXfmrs:
            DistAreaFlag1 = False
            DistAreaFlag2 = False
            SwitchArea['unaddressable_equipment'].extend(DistXfmrs)
            for i3 in range(len(DistXfmrs)):
                xfmr = DistXfmrs[i3]
                [LVTree, LVConnNodeDict] = spanning_tree.local_spanning_tree(ConnNodeDict, TerminalsDict, NodeList, TermList, EquipDict, 'TransformerTank', [xfmr],  MVBaseTree, 'all')
                LVTreeKeys = LVTree[xfmr]
                if LVTreeKeys: 
                    LVTreeKeys.pop(0)
                    for i4 in range(len(LVTreeKeys)):
                        lvnode = LVTreeKeys[i4]
                        DistAreaFlag2 = True
                        ProcessedNodes.append(lvnode)
                        
                        print('lvnode ', ConnNodeDict[lvnode]['name'], lvnode)
                        DistArea['distribution_transformer'] = [xfmr]
 #                       DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['SynchronousMachine'])
  #                      DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['PowerElectronicsConnection'])
   #                     DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['EnergyConsumer'])
    #                    DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['House'])
     #                   
      #                  DistArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['ACLineSegment'])
       #                 DistArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['Measurement'])
        #                DistArea['connectivity_node'].append(lvnode)
                        DistArea['connectivity_node'].append(ConnNodeDict[lvnode]['name'])
         #               SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['ACLineSegment'])
          #              SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['Measurement'])
                    
                #if DistArea['connectivity_node']: SwitchArea['secondary_areas'].append((DistArea.copy()))
    
            if DistAreaFlag2: 
                print('dist area', DistArea['connectivity_node'] )
                SwitchArea['secondary_areas'].append((DistArea.copy()))

    if DistAreaFlag1: 
        print('dist area', DistArea['connectivity_node'] )
        SwitchArea['secondary_areas'].append((DistArea.copy()))
        
    if SwitchArea['boundary_switches']: DistAppStruct['feeders'][model_mrid]['switch_areas'].append(dict(SwitchArea))

        #DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(SwitchArea['unaddressable_equipment'])
        
AllNodes = list(ConnNodeDict.keys())
MissingNodes = list(set(AllNodes).difference(ProcessedNodes))
for i5 in range(len(MissingNodes)):
    node = MissingNodes[i5]
    #DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['SynchronousMachine'])
    #DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['PowerElectronicsConnection'])
    #DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['LinearShuntCompensator'])
    #DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['RatioTapChanger'])
    #DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['ACLineSegment'])
    #DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['PowerTransformer'])
    #DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['TransformerTank'])
    #DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['Measurement'])
    DistAppStruct['feeders'][model_mrid]['connectivity_node'].append(ConnNodeDict[node]['name'])
    

In [ ]:
DistAppStruct

In [ ]:
EquipDict['LoadBreakSwitch']

In [ ]:
MVConnNodeDict

In [ ]:
MVTerminalsDict[MVTermList[6]]

In [ ]:
MVConnNodeDict[MVNodeList[5]]

In [ ]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

object_mrid = "_2858B6C2-0886-4269-884C-06FA8B887319"

message = {
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "modelId": '_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62',
    "objectId": '_DBFA8EFA-B397-40DA-9642-60E8DE4B3471'
}

data = gapps.get_response(topic, message)

In [ ]:
data['data']

In [ ]:
for i0 in range(len(SwitchKeys)):
    print(SwitchDict[SwitchKeys[i0]]['name'])
    for i8 in range(len(MVTrees[SwitchKeys[i0]])):
        
        print(ConnNodeDict[MVTrees[SwitchKeys[i0]][i8]]['name'])

In [ ]:
import sys
sys.getsizeof(DistAppStruct)

In [ ]:
DistAppStruct

In [ ]:
DistArea


In [ ]:
TreeKeys = list(MVTrees.keys())
for i10 in range(len(TreeKeys)):
    message = {
        "requestType": "QUERY_OBJECT_DICT",
        "resultFormat": "JSON",
        "modelId": '_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62',
        "objectId": TreeKeys[i10]
    }

    data=gapps.get_response(topic, message)
    print()
    print(data['data'][0]['IdentifiedObject.name'])
    for i11 in range(len(MVTrees[TreeKeys[i10]])):
        print(ConnNodeDict[MVTrees[TreeKeys[i10]][i11]]['name'])

In [ ]:
TreeKeys = list(MVTrees.keys())
for i10 in range(len(TreeKeys)):
    
    for i11 in range(len(MVTrees[TreeKeys[i10]])):
        print(ConnNodeDict[MVTrees[TreeKeys[i10]][i11]]['name'])